PART III Prep

https://docs.databricks.com/en/sql/language-manual/index.html

How to use Hive Variables to substitute in string values derived from the account email of the current user

In [ ]:
SELECT "${da.db_name}" AS db_name,
       "${da.paths.working_dir}" AS working_dir


CREATE DATABASE IF NOT EXISTS ${da.db_name}_default_location;
CREATE DATABASE IF NOT EXISTS ${da.db_name}_custom_location LOCATION '${da.paths.working_dir}/_custom_location.db';

Identify where Delta Lake provides ACID transactions

Merge update does this AND Delta Lake guarantees that any read against a table will return the most recent version of the table. The purpose of merge is to consolidate update, insert, and other manipulations into one command much like an upsert

In [ ]:
MERGE INTO students b
USING updates u
ON b.id=u.id
WHEN MATCHED AND u.type = "update"
  THEN UPDATE SET *
WHEN MATCHED AND u.type = "delete"
  THEN DELETE
WHEN NOT MATCHED AND u.type = "insert"
  THEN INSERT *

Create a managed table (Delta by default as it is backed by delta lake) Metadata + Data Management

Managed tables manage underlying data files alongside the metastore registration. Databricks recommends that you use managed tables whenever you create a new table. Unity Catalog managed tables are the default when you create tables in Databricks. They always use Delta Lake. See Work with managed tables.

By default, managed tables in a database without the location specified will be created in the dbfs:/user/hive/warehouse/<database_name>.db/ directory.

We can see that, as expected, the data and metadata for our Delta Table are stored in that location.



In [ ]:
/*full specification and Unity Catalog Approach*/
CREATE TABLE catalog_name.schema_name.table_name
  (width INT, length INT, height INT);

INSERT INTO managed_default
VALUES (3 INT, 2 INT, 1 INT)

/*HIVE approach, this will create the table in the default location under dbfs:/user/hive/warehouse/ remember the variables up top*/
CREATE DATABASE IF NOT EXISTS ${da.db_name}_default_location;
CREATE DATABASE IF NOT EXISTS ${da.db_name}_custom_location LOCATION '${da.paths.working_dir}/_custom_location.db';

Create an External Table (Metadata managed by Databricks only)


In [ ]:
/*Exammple 1 with HIVE*/
CREATE TABLE my_external_table
  (width INT, length INT, height INT)
LOCATION 'dbfs:/mnt/demo/external_default';

  
INSERT INTO external_default
VALUES (3 INT, 2 INT, 1 INT)

hive_metastore.default.managed_default

In [ ]:
/*Example 2 with Unity Catalog*/

CREATE TABLE my_external_table
    (width INT. length INT, height INT)
LOCATION 'abfss://<bucket-path>/<table-directory>';

Explore table Metadata

In [ ]:
/*basic metadata*/
DESCRIBE TABLE customer;

DESCRIBE TABLE extended my_table_name;



Return information about a schema: 

Returns information about schema, partitioning, table size, and so on

In [ ]:
/*Managed Table*/
DESCRIBE DETAIL [schema_name].table_name

DESCRIBE EXTENDED students_table

/*External Location*/
DESCRIBE EXTERNAL LOCATION table_location_name;

/*Show external locations*/
SHOW EXTERNAL LOCATION

Identify the location of a table

Use the SHOW table command

In [ ]:
USE <your_database_name>

/*show tables*/
SHOW TABLES

/*show tables in global temp*/
SHOW TABLES IN global_temp

Inspect the directory structure of Delta Lake files

In [ ]:
display(dbutils.fs.ls(f"{DA.paths.user_db}/students"))

#using DB Utils
fs ls 'dbfs:/user/hive/warehouse/employees'


Identify who has written previous versions of a table

In [ ]:
DESCRIBE HISTORY table_name;

Review a history of table transactions

In [ ]:
DESCRIBE HISTORY my_table_name

Roll back a table to a previous version

In [ ]:
/*Option 1*/
RESTORE TABLE employees TO VERSION AS OF 5

/*Option 2*/
RESTORE TABLE table_name TO TIMESTAMP AS OF 'yyyy-MM-dd HH:mm:ss';

Identify that a table can be rolled back to a previous version

In [ ]:
DESCRIBE HISTORY table_name;

Query a specific version of a table

In [ ]:
SELECT * 
FROM employees VERSION AS OF 4

Identify CTAS as a solution and create a table using CTAS




CTAS statements automatically infer schema information from query results and do not support manual schema declaration.

This means that CTAS statements are useful for external data ingestion from sources with well-defined schema, such as Parquet files and tables.

CTAS statements also do not support specifying additional file options.

We can see how this would present significant limitations when trying to ingest data from CSV files.

In [ ]:
/*FROM A TABLE*/
CREATE OR REPLACE TABLE my_table AS
SELECT * FROM my_other_table


/*FROM A FILE*/
CREATE OR REPLACE TABLE sales_unparsed AS
SELECT * FROM csv.`${da.paths.datasets}/raw/sales-csv/`;
SELECT * FROM sales_unparsed;

Create a generated column

DE 4.3

In [ ]:
CREATE OR REPLACE TABLE purchase_dates (
  id STRING, 
  transaction_timestamp STRING, 
  price STRING,
  date DATE GENERATED ALWAYS AS (
    cast(cast(transaction_timestamp/1e6 AS TIMESTAMP) AS DATE))
    COMMENT "generated based on `transactions_timestamp` column")

Add a table comment

DE 4.3

In [ ]:
CREATE OR REPLACE TABLE users_pii
COMMENT "Contains PII"
LOCATION "${da.paths.working_dir}/tmp/users_pii"
PARTITIONED BY (first_touch_date)
AS
  SELECT *, 
    cast(cast(user_first_touch_timestamp/1e6 AS TIMESTAMP) AS DATE) first_touch_date, 
    current_timestamp() updated,
    input_file_name() source_file
  FROM parquet.`${da.paths.datasets}/raw/users-historical/`;
  
SELECT * FROM users_pii;

Use CREATE OR REPLACE TABLE and INSERT OVERWRITE

DE 4.3 and 4.4

We can use overwrites to atomically replace all of the data in a table. There are multiple benefits to overwriting tables instead of deleting and recreating tables:

Overwriting a table is much faster because it doesn’t need to list the directory recursively or delete any files.
The old version of the table still exists; can easily retrieve the old data using Time Travel.
It’s an atomic operation. Concurrent queries can still read the table while you are deleting the table.
Due to ACID transaction guarantees, if overwriting the table fails, the table will be in its previous state.
Spark SQL provides two easy methods to accomplish complete overwrites.

Some students may have noticed previous lesson on CTAS statements actually used CRAS statements (to avoid potential errors if a cell was run multiple times).

CREATE OR REPLACE TABLE (CRAS) statements fully replace the contents of a table each time they execute.

INSERT OVERWRITE:

Can only overwrite an existing table, not create a new one like our CRAS statement
Can overwrite only with new records that match the current table schema -- and thus can be a "safer" technique for overwriting an existing table without disrupting downstream consumers
Can overwrite individual partitions

In [ ]:
CREATE OR REPLACE TABLE my_table

/*INSERT OVERWRITE*/
INSERT OVERWRITE sales
SELECT * FROM parquet.`${da.paths.datasets}/raw/sales-historical/`

Identify a scenario in which MERGE should be used

Upsert & to guarantee Atomicity

DE 4.4

In [ ]:
MERGE INTO target a
USING source b
ON {merge_condition}
WHEN MATCHED THEN {matched_action}
WHEN NOT MATCHED THEN {not_matched_action}


MERGE INTO users a
USING users_update b
ON a.user_id = b.user_id
WHEN MATCHED AND a.email IS NULL AND b.email IS NOT NULL THEN
  UPDATE SET email = b.email, updated = b.updated
WHEN NOT MATCHED THEN INSERT *

Identify MERGE as a command to deduplicate data upon writing

Identify why a COPY INTO statement is not duplicating data in the target table

DE 4.4


COPY INTO provides SQL engineers an idempotent option to incrementally ingest data from external systems.

Note that this operation does have some expectations:

Data schema should be consistent
Duplicate records should try to be excluded or handled downstream
This operation is potentially much cheaper than full table scans for data that grows predictably.

While here we'll show simple execution on a static directory, the real value is in multiple executions over time picking up new files in the source automatically.

In [ ]:
COPY INTO sales
FROM "${da.paths.datasets}/raw/sales-30m"
FILEFORMAT = PARQUET

Use COPY INTO to insert data

In [ ]:
COPY INTO sales
FROM "${da.paths.datasets}/raw/sales-30m"
FILEFORMAT = PARQUET

Identify the components necessary to create a new DLT pipeline

In [ ]:
/*Create Bronze table*/

CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_raw
COMMENT "The raw sales orders, ingested from /databricks-datasets."
AS SELECT * FROM cloud_files("/databricks-datasets/retail-org/sales_orders/", "json", map("cloudFiles.inferColumnTypes", "true"))

/*Create a Silver Table*/
CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_cleaned(
  CONSTRAINT valid_order_number EXPECT (order_number IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "The cleaned sales orders with valid order_number(s) and partitioned by order_datetime."
AS
  SELECT f.customer_id, f.customer_name, f.number_of_line_items, 
         timestamp(from_unixtime((cast(f.order_datetime as long)))) as order_datetime, 
         date(from_unixtime((cast(f.order_datetime as long)))) as order_date, 
         f.order_number, f.ordered_products, c.state, c.city, c.lon, c.lat, c.units_purchased, c.loyalty_segment
  FROM STREAM(LIVE.sales_orders_raw) f
  LEFT JOIN LIVE.customers c
    ON c.customer_id = f.customer_id
    AND c.customer_name = f.customer_name

/*Create a Gold table*/
CREATE OR REFRESH LIVE TABLE sales_order_in_la
COMMENT "Sales orders in LA."
AS
  SELECT city, order_date, customer_id, customer_name, ordered_products_explode.curr, 
         sum(ordered_products_explode.price) as sales, 
         sum(ordered_products_explode.qty) as quantity, 
         count(ordered_products_explode.id) as product_count
  FROM (SELECT city, order_date, customer_id, customer_name, explode(ordered_products) as ordered_products_explode
        FROM LIVE.sales_orders_cleaned 
        WHERE city = 'Los Angeles')
  GROUP BY order_date, city, customer_id, customer_name, ordered_products_explode.curr

Identify which source location is utilizing Auto Loader

DE 6.0 Series

Explain change data capture and the behavior of APPLY CHANGES INTO